In [1]:
#old data, no generator, less crop doesn't work

In [2]:
import numpy as np
import pandas as pd
import cv2
import math
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Lambda, Flatten
from keras.layers.convolutional import Conv2D, Cropping2D, MaxPooling2D
from keras.layers.advanced_activations import ELU
from keras import backend as K
from keras.optimizers import SGD
from tqdm import tqdm
import random
from sklearn.utils import shuffle
import sys

#print(K.image_data_format())
#'channels_first'
#K.set_image_data_format('channels_last')
#print(K.image_data_format())



Using TensorFlow backend.


In [3]:
split_fraction = 25000

all_data = pd.read_csv("mydata2/driving_log.csv", names = ["center", "left", "right", "Steering", "throtle", "something","speed"])
all_data_shuffled = shuffle(all_data)

print (all_data.shape)

training_data = all_data_shuffled[:split_fraction]
validation_data = all_data_shuffled[split_fraction:]

print (training_data.shape)
print (validation_data.shape)



(27242, 7)
(25000, 7)
(2242, 7)


In [4]:
#generator
def driving_generator(working_data):
    # creating generator that gives always balanced batches
    
    pandas_batch = pd.DataFrame()
    images_list = []
    new_list = shuffle(working_data)
    
    max_steering = new_list.iloc[:,[3]].max()
    max_steering = max_steering.as_matrix()[0]

    min_steering = new_list.iloc[:,[3]].min()
    min_steering = min_steering.as_matrix()[0]
    
    buckets = 32
    
    print("max", max_steering)
    print("min", min_steering)

    step = abs((min_steering - max_steering) / buckets)
    print ("step", step)

    steering_range_low = min_steering
    steering_range_high = 0
    
    while True:
        steering_range_low = min_steering
        steering_range_high = 0
        for i in range(buckets):

                steering_range_high = steering_range_low + step
                #print ("high",steering_range_high,"low", steering_range_low)
                bucket_list = new_list[(new_list.Steering >= steering_range_low) & (new_list.Steering < steering_range_high)]
                bucket_quantity = len(bucket_list)
                if bucket_quantity >0:
                    chosen = bucket_list.sample(1)
                    #print (chosen)
                    steering_range_low += step
                    pandas_batch = pandas_batch.append(chosen)
        #print (pandas_batch.shape)

        names = pandas_batch.iloc[:,[0]].as_matrix()[:,0]
        steering = pandas_batch.iloc[:,[3]].as_matrix()[:,0]
        #print ("names",names)
        #print ("steering", steering)

        for name in names:
            #print (name)
            #print (name.split('\\'))
            if name.startswith('/'):
                #print (name[0].split('/'))
                splited_name = name.split('/')
                path = 'mydata2/IMG/'+splited_name[9]
                #print ("media",path)
            else:
                #print (name.split('\\'))
                splited_name = name.split('\\')
                path = 'mydata2/IMG/'+splited_name[6]
                #print (path)

            image = cv2.imread(path)
            image = image[...,::-1] #conversion to rgb
            #plt.imshow(image)
            #plt.show()
            #print(image)
            images_list.append(image)

        X_train = np.array(images_list)
        y_train = np.expand_dims(steering, axis=1)

        pandas_batch = pd.DataFrame()
        images_list = []
        #print (X_train, y_train)
        #print ("\n\n")
        # unpack batch properly
        yield (X_train, y_train)

#franek = driving_generator(training_data)
#next(franek)
#next(franek)
#raise SystemExit()


In [5]:



def nvidia_model():
    model = Sequential()
    nv_model = Sequential()
    nv_model.add (Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
    nv_model.add (Cropping2D(cropping=((20,20), (0,0)), input_shape=(160,320,3)))
    
    nv_model.add (Conv2D(24, (5,5), strides=1, padding="valid"))
    nv_model.add (ELU())
    nv_model.add ((MaxPooling2D(pool_size=(2,2))))

    nv_model.add (Conv2D(36, (5,5), strides=1, padding="valid"))
    nv_model.add (ELU())
    nv_model.add ((MaxPooling2D(pool_size=(2,2))))
        
    nv_model.add (Conv2D(48, (5,5), strides=1, padding="valid"))
    nv_model.add (ELU())
    nv_model.add ((MaxPooling2D(pool_size=(2,2))))
    
    nv_model.add (Conv2D(64, (3,3), strides=1, padding="valid"))
    nv_model.add (ELU())
    
    nv_model.add (Conv2D(64, (3,3), strides=1, padding="valid"))
    nv_model.add (ELU())
    
     
    nv_model.add(Flatten())
    nv_model.add(Dense(1164))
    nv_model.add(ELU())
    nv_model.add(Dense(100))
    nv_model.add(ELU())
    nv_model.add(Dense(50))
    nv_model.add(ELU())
    nv_model.add(Dense(10))
    nv_model.add(ELU())
    
    nv_model.add(Dense(1, activation="linear"))
    nv_model.compile(optimizer="adam", metrics=['accuracy'], loss="mse")
    return nv_model

my_model = nvidia_model()


# zmienic zeby validation pracowalo na valid data
training_generator = driving_generator(training_data)
validation_generator = driving_generator(validation_data)

#my_model.fit(X_train, y_train,  validation_data=(X_valid,y_valid), shuffle=True, batch_size=32, epochs=5, verbose=1)
#model.fit_generator(train_generator, samples_per_epoch= /
          #  len(train_samples), validation_data=validation_generator, /
          #  nb_val_samples=len(validation_samples), nb_epoch=3)
        
my_model.fit_generator(training_generator,  validation_data=validation_generator, validation_steps=(validation_data.shape[0] // 32), steps_per_epoch=1000, epochs=4, verbose=1)
my_model.save("model.h5")
#score = model.evaluate(x_test, y_test, batch_size=32)




Epoch 1/4
max 0.3838028
min -0.4047619
step 0.024642646875
 999/1000 [============================>.] - ETA: 0s - loss: 0.1120 - acc: 0.0021- Emax 0.3392018
min -0.3556548
step 0.02171426875
1000/1000 [==============================] - 183s - loss: 0.1118 - acc: 0.0021 - val_loss: 0.0010 - val_acc: 0.0000e+00
Epoch 2/4
1000/1000 [==============================] - 107s - loss: 8.5545e-04 - acc: 0.0022 - val_loss: 0.0015 - val_acc: 0.0000e+00
Epoch 3/4
1000/1000 [==============================] - 106s - loss: 5.8393e-04 - acc: 0.0021 - val_loss: 0.0013 - val_acc: 0.0000e+00
Epoch 4/4
1000/1000 [==============================] - 105s - loss: 4.7439e-04 - acc: 0.0018 - val_loss: 2.9895e-04 - val_acc: 0.0000e+00
